In [1]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

In [55]:
df = pd.read_csv('/Users/james/Downloads/train.csv')

A retail company “ABC Private Limited” wants to understand the customer purchase behaviour (specifically, purchase amount) against various products of different categories.

They have shared purchase summary of various customers for selected high volume products from last month.

The data set also contains customer demographics (age, gender, marital status, city_type, stay_in_current_city), product details (product_id and product category) and Total purchase_amount from last month.

In [60]:
## Competition data is usually pretty clean, but just in case. . . . 
## show columns with null values, should be only prod_cat 2 and 3

[i for i in df.columns if df[i].isnull().values.any()]

['Product_Category_2', 'Product_Category_3']

In [61]:
## Oh my god we dont have to handle null values, yes!

df[:10]

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871


In [62]:
df = df.drop(columns = ['User_ID', 'Product_ID'])

In [63]:
df['Prod_Cat'] = df[idents].apply(lambda x: '-'.join(x.values.astype(int).astype(str)), axis=1)

In [64]:
df = df.drop(columns = ['Product_Category_1','Product_Category_2','Product_Category_3'] )

In [65]:
df[:10]

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Purchase,Prod_Cat
0,F,0-17,10,A,2,0,8370,3-0-0
1,F,0-17,10,A,2,0,15200,1-6-14
2,F,0-17,10,A,2,0,1422,12-0-0
3,F,0-17,10,A,2,0,1057,12-14-0
4,M,55+,16,C,4+,0,7969,8-0-0
5,M,26-35,15,A,3,0,15227,1-2-0
6,M,46-50,7,B,2,1,19215,1-8-17
7,M,46-50,7,B,2,1,15854,1-15-0
8,M,46-50,7,B,2,1,15686,1-16-0
9,M,26-35,20,A,1,1,7871,8-0-0


In [68]:
## Create Ordinal variables, might switch these to straight dummy catergorical variables later 

df.Age.unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [67]:
age_mapper = {
    '0-17': 0,
    '18-25':1,
    '26-35':2,
    '36-45':3,
    '46-50':4,
    '51-55':5,
    '55+':6,
}

df['mapped_age'] = df['Age'].replace(age_mapper)

In [69]:
df.Stay_In_Current_City_Years.unique()

array(['2', '4+', '3', '1', '0'], dtype=object)

In [70]:
stay_mapper = {
    '0': 0,
    '1':1,
    '2':2,
    '3':3,
    '4+':4,
}

df['mapped_stay'] = df['Stay_In_Current_City_Years'].replace(stay_mapper)

In [72]:
df = df.drop(columns = ['Stay_In_Current_City_Years','Age'] )

In [73]:
df[:10]

,Gender,Occupation,City_Category,Marital_Status,Purchase,Prod_Cat,mapped_age,mapped_stay
0,F,10,A,0,8370,3-0-0,0,2
1,F,10,A,0,15200,1-6-14,0,2
2,F,10,A,0,1422,12-0-0,0,2
3,F,10,A,0,1057,12-14-0,0,2
4,M,16,C,0,7969,8-0-0,6,4
5,M,15,A,0,15227,1-2-0,2,3
6,M,7,B,1,19215,1-8-17,4,2
7,M,7,B,1,15854,1-15-0,4,2
8,M,7,B,1,15686,1-16-0,4,2
9,M,20,A,1,7871,8-0-0,2,1


In [74]:
## dummy cat variables

list_of_cats = [
    'Occupation',
    'City_Category',
    'Prod_Cat',
]

df = pd.get_dummies(df, columns = list_of_cats, drop_first = True)

In [75]:
df

,Gender,Marital_Status,Purchase,mapped_age,mapped_stay,Occupation_1,Occupation_2,Occupation_3,Occupation_4,Occupation_5,...,Prod_Cat_8-15-0,Prod_Cat_8-15-16,Prod_Cat_8-16-0,Prod_Cat_8-16-17,Prod_Cat_8-17-0,Prod_Cat_8-18-0,Prod_Cat_8-9-0,Prod_Cat_8-9-14,Prod_Cat_9-0-0,Prod_Cat_9-15-0
0,F,0,8370,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,F,0,15200,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,F,0,1422,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,F,0,1057,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M,0,7969,6,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550063,M,1,368,5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550064,F,0,371,2,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550065,F,1,137,2,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550066,F,0,365,6,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
# Up Next: Create baseline linreg up top, correlation coef, test existing features, feature creation, profit? 